In [1]:
from efficacy_utils import *

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


<IPython.core.display.Javascript object>

R[write to console]: Error in withVisible({ : 找不到对象'plot_attrition_df'

R[write to console]: 载入需要的程辑包：Matrix




Error in withVisible({ : 找不到对象'plot_attrition_df'


R[write to console]: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

R[write to console]: 
载入程辑包：‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step


R[write to console]: 
Please cite as: 


R[write to console]:  Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

R[write to console]:  R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


R[write to console]: Error in df$user : 类别为'closure'的对象不可以取子集

R[write to console]: 此外: 
R[write to console]: Warning messages:

R[write to console]: 1: 程辑包‘lme4’是用R版本3.6.2 来建造的 

R[write to console]: 2: 程辑包‘lmerTest’是用R版本3.6.2 来建造的 




Error in df$user : 类别为'closure'的对象不可以取子集
NULL


R[write to console]: Error in object[[i]] : 类别为'closure'的对象不可以取子集

R[write to console]: Error in df$domain : 类别为'closure'的对象不可以取子集




Error in object[[i]] : 类别为'closure'的对象不可以取子集

Error in df$domain : 类别为'closure'的对象不可以取子集


In [2]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [3]:
installs_with_experiment_vars = get_installs_with_experiment_vars()

In [4]:
print(len(installs_with_experiment_vars))

18195


In [5]:
intervention_name_to_difficulty = {
  'facebook/remove_news_feed': 1,
  #'facebook/prompt_reason': 2,
  'facebook/remove_clickbait': 1,
  #'internal/choose_difficulty',
  #'facebook/show_user_info_banner',
  'facebook/toast_notifications': 2,
  'facebook/feed_injection_timer': 1,
  'facebook/remove_comments': 2,
  'facebook/block_after_interval_per_visit': 2,
  'facebook/show_user_info_interstitial': 3,
  'facebook/close_tab_timer': 3,
  'facebook/scroll_blocker': 3,
  'facebook/make_user_wait': 2,
  'facebook/rich_notifications': 1,
  'facebook/show_timer_banner': 1,
}

session_info_list = []

intervention_set = set()
difficulty_to_session_lengths = {}
for install in installs_with_experiment_vars: #[:100]:
  domain_to_intervention_to_session_lengths = get_domain_to_intervention_to_session_lengths_for_install(install)
  for domain,intervention_to_session_lengths in domain_to_intervention_to_session_lengths.items():
    if domain != 'www.facebook.com':
      continue
    for intervention,session_lengths in intervention_to_session_lengths.items():
      difficulty = intervention_name_to_difficulty.get(intervention)
      if difficulty == None:
        continue
      if difficulty not in difficulty_to_session_lengths:
        difficulty_to_session_lengths[difficulty] = []
      for session_length in session_lengths:
        difficulty_to_session_lengths[difficulty].append(session_length)
        session_info_list.append({
          'user': install,
          'difficulty': difficulty,
          'logtime': math.log(session_length),
        })

In [6]:
from browser_libs import convert_list_of_dicts_into_dataframe

df = convert_list_of_dicts_into_dataframe(session_info_list)

In [7]:
%Rpush df

In [8]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
library(stargazer)

In [9]:
%%R

df$user <- as.factor(df$user)
df$difficulty <- factor(df$difficulty, levels = c(1, 2, 3))
df$logtime <- as.numeric(df$logtime)
summary(df)

[[1]]
[1] "05b3c37c2a72180d4cf5e590:  53190  "

[[2]]
[1] "123ce1aebe3fee618bbc8e73:  34833  "

[[3]]
[1] "3e3ac64e84482ca34bc271e9:  28859  "

[[4]]
[1] "a85b38dea50b24bc26604e2d:  22596  "

[[5]]
[1] "feb60418db5ca1238fd322cf:  21601  "

[[6]]
[1] "34402cda1da6dedf07eaaaa5:  20904  "

[[7]]
[1] "(Other)                 :3427678  "

[[8]]
[1] "1:2197124  "

[[9]]
[1] "2:1093239  "

[[10]]
[1] "3: 319298  "

[[11]]
<CHARSXP: NA>

[[12]]
<CHARSXP: NA>

[[13]]
<CHARSXP: NA>

[[14]]
<CHARSXP: NA>

[[15]]
[1] "Min.   : 0.000  "

[[16]]
[1] "1st Qu.: 3.912  "

[[17]]
[1] "Median : 5.198  "

[[18]]
[1] "Mean   : 5.105  "

[[19]]
[1] "3rd Qu.: 6.375  "

[[20]]
[1] "Max.   :11.662  "

[[21]]
<CHARSXP: NA>



In [10]:
%%R

results <- lmer(logtime ~ difficulty + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
show(results)
show(summary(results))
class(results) <- "lmerMod"
stargazer(results)

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: logtime ~ difficulty + (1 | user)
   Data: df
REML criterion at convergence: 13360619
Random effects:
 Groups   Name        Std.Dev.
 user     (Intercept) 1.021   
 Residual             1.534   
Number of obs: 3609661, groups:  user, 8518
Fixed Effects:
(Intercept)  difficulty2  difficulty3  
    4.42003      0.01395     -0.14759  
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: logtime ~ difficulty + (1 | user)
   Data: df

REML criterion at convergence: 13360619

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2982 -0.5827  0.1321  0.7084  3.6775 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.042    1.021   
 Residual             2.354    1.534   
Number of obs: 3609661, groups:  user, 8518

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  4.420e+00  1.213e-02  7.586e+03 364.553  < 2e-

In [11]:
import numpy as np

In [12]:
difficulty_to_median_session_lengths = {}
for difficulty,session_lengths in difficulty_to_session_lengths.items():
  difficulty_to_median_session_lengths[difficulty] = np.mean(session_lengths)


In [13]:
from plot_utils import *

In [14]:
plot_dict_as_bar(difficulty_to_median_session_lengths)

In [15]:
print(intervention_set)

set()
